In [0]:
%sql
USE CATALOG workspace;

USE SCHEMA sales_bronze;

In [0]:
%sql
DROP TABLE IF EXISTS sales_bronze.current_employee_bronze;

CREATE TABLE sales_bronze.current_employee_bronze
(
  ID INT,
  FirstName STRING,
  Country STRING,
  Role STRING
)

In [0]:
spark.sql(f'''
COPY INTO sales_bronze.current_employee_bronze
FROM '/Volumes/workspace/sales_bronze/employee/'
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true', 'InferSchema' = 'true')
          
 '''
).display()

num_affected_rows,num_inserted_rows,num_skipped_corrupt_files
0,0,0


In [0]:
%sql
select * from sales_bronze.current_employee_bronze

ID,FirstName,Country,Role
5555,Alex,USA,Instructor
6666,Sanjay,India,Instructor
7777,Roshan,USA,Instructor
8888,Gareth,France,Instructor
3333,Zebi,Pakistan,Administrator
4444,Peter,USA,Developer


In [0]:
%sql
CREATE OR REPLACE TABLE sales_bronze.current_employee_silver
AS
select ID, FirstName, Country,
upper(Role) as Role, 
current_timestamp() as CurrentTimestamp, 
date(CurrentTimestamp) as CurrentDate
FROM sales_bronze.current_employee_bronze

num_affected_rows,num_inserted_rows


In [0]:
%sql
select * from sales_bronze.current_employee_silver

ID,FirstName,Country,Role,CurrentTimestamp,CurrentDate
5555,Alex,USA,INSTRUCTOR,2025-12-08T16:44:59.267Z,2025-12-08
6666,Sanjay,India,INSTRUCTOR,2025-12-08T16:44:59.267Z,2025-12-08
7777,Roshan,USA,INSTRUCTOR,2025-12-08T16:44:59.267Z,2025-12-08
8888,Gareth,France,INSTRUCTOR,2025-12-08T16:44:59.267Z,2025-12-08
3333,Zebi,Pakistan,ADMINISTRATOR,2025-12-08T16:44:59.267Z,2025-12-08
4444,Peter,USA,DEVELOPER,2025-12-08T16:44:59.267Z,2025-12-08


In [0]:
%sql
CREATE OR REPLACE TEMP VIEW temp_view_total_roles AS
SELECT Role, count(*) FROM sales_bronze.current_employee_silver
GROUP BY Role;

SELECT * FROM temp_view_total_roles

Role,count(*)
INSTRUCTOR,4
ADMINISTRATOR,1
DEVELOPER,1


In [0]:
%sql
CREATE OR REPLACE TABLE sales_bronze.total_roles_gold
(Role STRING,
TotalEmployees INT);

In [0]:
%sql
INSERT OVERWRITE TABLE sales_bronze.total_roles_gold
SELECT * FROM temp_view_total_roles;

num_affected_rows,num_inserted_rows
3,3


In [0]:
%sql
select * from sales_bronze.total_roles_gold

Role,TotalEmployees
ADMINISTRATOR,1
INSTRUCTOR,4
DEVELOPER,1


In [0]:
%sql
DESCRIBE HISTORY sales_bronze.total_roles_gold

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
2,2025-12-08T17:00:07.000Z,70858962676464,abhilashaalexander93@gmail.com,WRITE,"Map(mode -> Overwrite, statsOnLoad -> true, partitionBy -> [])",null,List(2351183914159033),1208-163612-ayo3wv0m-v2n,1,WriteSerializable,false,"Map(numFiles -> 1, numRemovedFiles -> 1, numRemovedBytes -> 953, numDeletionVectorsRemoved -> 0, numOutputRows -> 3, numOutputBytes -> 953)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
1,2025-12-08T16:51:32.000Z,70858962676464,abhilashaalexander93@gmail.com,WRITE,"Map(mode -> Overwrite, statsOnLoad -> true, partitionBy -> [])",null,List(2351183914159033),1208-163612-ayo3wv0m-v2n,0,WriteSerializable,false,"Map(numFiles -> 1, numRemovedFiles -> 0, numRemovedBytes -> 0, numDeletionVectorsRemoved -> 0, numOutputRows -> 3, numOutputBytes -> 953)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
0,2025-12-08T16:49:11.000Z,70858962676464,abhilashaalexander93@gmail.com,CREATE OR REPLACE TABLE,"Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {""delta.parquet.compression.codec"":""zstd"",""delta.enableDeletionVectors"":""true"",""delta.writePartitionColumnsToParquet"":""true"",""delta.enableRowTracking"":""true"",""delta.rowTracking.materializedRowCommitVersionColumnName"":""_row-commit-version-col-b1accc6c-cab8-4260-b5b9-daa938440af7"",""delta.rowTracking.materializedRowIdColumnName"":""_row-id-col-10f2dae9-79bc-4aa4-9e39-ad7e1345f831""}, statsOnLoad -> false)",null,List(2351183914159033),1208-163612-ayo3wv0m-v2n,null,WriteSerializable,true,Map(),null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
